In [ ]:
import os
import sys
import json
import pickle as pkl
import numpy as np
import pandas as pd
import arviz as az
import pymc3 as pm
import theano
import theano.tensor as T
from sklearn.model_selection import train_test_split

In [ ]:
sys.path.append("../")
from mcmc_utils import *

In [ ]:
# read data
fp = os.path.abspath('../dataset/model_set.csv')
raw_df = pd.read_csv(fp)

# feature engineering
raw_df["LogSalePrice"] = np.log(raw_df.SalePrice)
raw_df.CentralAir = [1 if i == "Y" else 0 for i in raw_df.CentralAir]
raw_df["StoneVnr"] = [1 if i == "Stone" else 0 for i in raw_df.MasVnrType]
kitch_qual_conv = {"Ex": 3, "Gd": 2, "TA": 1, "Fa": 0}
raw_df.KitchenQual = [kitch_qual_conv[i] for i in raw_df.KitchenQual]
raw_df.YrSold = raw_df.YrSold - raw_df.YrSold.min()  # years from 2006
raw_df.YearBuilt = raw_df.YearBuilt - raw_df.YearBuilt.min()  # years from 1872
raw_df.YearRemodAdd = raw_df.YearRemodAdd - raw_df.YearRemodAdd.min()  # years from 1950
Neighborhoods = raw_df.Neighborhood.unique()
NbdLookup = dict(zip(Neighborhoods, range(Neighborhoods.size)))
raw_df["NeighborhoodCode"] = raw_df.Neighborhood.replace(NbdLookup)

# drop unecessary cols
d_cols = ["Utilities"]
raw_df.drop(columns=d_cols, inplace=True)

### data preparation and formatting ###

# design matix
covariates = ("1stFlrSF", "LotArea", "StoneVnr", "KitchenQual")
y = raw_df.LogSalePrice
X = raw_df.loc[:, covariates]
X_nbd = raw_df.loc[:, "NeighborhoodCode"]
n_nbd = Neighborhoods.size
n, p = X.shape

# train-test split
train_idx, test_idx = train_test_split(range(n),
                                       test_size=0.2,
                                       random_state=1)
X_train = X.iloc[train_idx, :].reset_index(drop=True)
X_nbd_train = X_nbd.iloc[train_idx].reset_index(drop=True)
X_test = X.iloc[test_idx, :].reset_index(drop=True)
X_nbd_test = X_nbd.iloc[test_idx].reset_index(drop=True)
y_train = y.iloc[train_idx].reset_index(drop=True)
y_test = y.iloc[test_idx].reset_index(drop=True)

### modeling ###
hp_model = pm.Model()
#prev_trace = pm.load_trace('.pymc_1.trace', model=hp_model)

Ip_mat = np.eye(p)
zp_vec = np.zeros(p)

# specify model and perform sampling
with hp_model:
    X_train_data = pm.Data("X_train_data", X_train)
    nbd_idx = pm.Data("nbd_idx", X_nbd_train)
    y_train_data = pm.Data("y_train_data", y_train)
    # hyper priors
    chol, corr, stds = pm.LKJCholeskyCov("Omega", n=p, eta=1.,
                                         sd_dist=pm.HalfStudentT.dist(sigma=0.5,
                                                                      nu=1.),
                                         compute_corr=True)
    cov = pm.Deterministic("cov", chol.dot(chol.T))
    tau_alpha = pm.HalfStudentT("tau_alpha", sigma=0.5, nu=1.)
    alpha = pm.Normal("alpha", mu=12., sigma=0.5)
    # priors
    alpha_nbd = pm.Normal("alpha_nbd",
                          mu=alpha,
                          sigma=tau_alpha,
                          shape=(n_nbd,))
    beta_uc = pm.MvNormal("beta_uc", mu=zp_vec, cov=Ip_mat, shape=(p,))
    beta = pm.Deterministic("beta", cov.dot(beta_uc))
    sigma = pm.HalfStudentT("sigma", sigma=0.5, nu=1.)
    # likelihood
    Ey_x = T.add(alpha_nbd[nbd_idx], X_train_data.dot(beta))  # E[Y|X]
    y_obs = pm.Normal("y_obs", mu=Ey_x, sigma=sigma, observed=y_train_data)

In [ ]:
pm.model_to_graphviz(hp_model)

In [ ]:
trace = pm.load_trace("../.pymc_3.trace/", model=hp_model)
trace = az.from_pymc3(trace, model=hp_model)
trace_df = trace.to_dataframe()

#### if loading from netCDF
trace = az.from_netcdf("./inf_data.nc")

trace_df = trace.to_dataframe()

In [ ]:
for i, c in enumerate(covariates):
    print(c)
    bi_rhat = calc_Rhat(trace.posterior.beta[0,:,i].values,
                        trace.posterior.beta[1,:,i].values,
                        trace.posterior.beta[2,:,i].values)
    print(f"rhat for {c} coef: {bi_rhat}")
    plot_trace(trace.posterior.beta[0,:,i],
               trace.posterior.beta[1,:,i],
               trace.posterior.beta[2,:,i])
    plot_posterior_dists(trace.posterior.beta[0,:,i],
                         trace.posterior.beta[1,:,i],
                         trace.posterior.beta[2,:,i])
    print("--------")

In [ ]:
for i, c in enumerate(covariates):
    print(c)
    bi_rhat = calc_Rhat(trace.posterior.Omega_stds[0,:,i].values,
                        trace.posterior.Omega_stds[1,:,i].values,
                        trace.posterior.Omega_stds[2,:,i].values)
    print(f"rhat for {c} coef std: {bi_rhat}")
    plot_trace(trace.posterior.Omega_stds[0,:,i].values,
               trace.posterior.Omega_stds[1,:,i].values,
               trace.posterior.Omega_stds[2,:,i].values)
    plot_posterior_dists(trace.posterior.Omega_stds[0,:,i].values,
                         trace.posterior.Omega_stds[1,:,i].values,
                         trace.posterior.Omega_stds[2,:,i].values)
    print("--------")

In [ ]:
alpha_rhat = calc_Rhat(trace.posterior.alpha[0,:].values,
                       trace.posterior.alpha[1,:].values,
                       trace.posterior.alpha[2,:].values)
print(f"Rhat for population alpha {alpha_rhat}")
plot_trace(trace.posterior.alpha[0,:].values,
           trace.posterior.alpha[1,:].values,
           trace.posterior.alpha[2,:].values)
plot_posterior_dists(trace.posterior.alpha[0,:].values,
                     trace.posterior.alpha[1,:].values,
                     trace.posterior.alpha[2,:].values)

In [ ]:
for i in range(n_nbd):
    print(Neighborhoods[i])
    print(calc_Rhat(trace.posterior.alpha_nbd[0,:,i].values,
                    trace.posterior.alpha_nbd[1,:,i].values,
                    trace.posterior.alpha_nbd[2,:,i].values))
    plot_trace(trace.posterior.alpha_nbd[0,:,i].values,
               trace.posterior.alpha_nbd[1,:,i].values,
               trace.posterior.alpha_nbd[2,:,i].values)
    plot_posterior_dists(trace.posterior.alpha_nbd[0,:,i].values,
                         trace.posterior.alpha_nbd[1,:,i].values,
                         trace.posterior.alpha_nbd[2,:,i].values)
    print("-------")

In [ ]:
for c in trace_df.columns:
    print(c)

In [ ]:
for nbd in range(n_nbd):
    plt.scatter(trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', 'alpha')],
                trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', f'alpha_nbd[{nbd}]', nbd)],
                color="black", alpha=0.2)
    plt.scatter(trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', 'alpha')],
                trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', f'alpha_nbd[{nbd}]', nbd)],
                color="limegreen", alpha=0.2)
    plt.title(f"Neighborhood {Neighborhoods[nbd]}")
    plt.show()
    print("--------")

In [ ]:
n1 = 0
n2 = 3
plt.scatter(trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', f'Omega_stds[{n1}]', n1)],
            trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', f'Omega_stds[{n2}]', n2)],
            color="black", alpha=0.2)
plt.scatter(trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', f'Omega_stds[{n1}]', n1)],
            trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', f'Omega_stds[{n2}]', n2)],
            color="limegreen", alpha=0.2)
plt.show()

In [ ]:
n = 0
plt.scatter(trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', f'beta[{n}]', n)],
            trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', f'Omega_stds[{n}]', n)],
            color="black", alpha=0.2)
plt.scatter(trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', f'beta[{n}]', n)],
            trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', f'Omega_stds[{n}]', n)],
            color="limegreen", alpha=0.2)
plt.xlabel(f"beta {n}")
plt.ylabel(f"beta_std {n}")
plt.show()

In [ ]:
plt.scatter(trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', 'alpha')],
            trace_df[~trace_df[('sample_stats', 'diverging')]][('posterior', 'tau_alpha')],
            color="black", alpha=0.2)
plt.scatter(trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', 'alpha')],
            trace_df[trace_df[('sample_stats', 'diverging')]][('posterior', 'tau_alpha')],
            color="limegreen", alpha=0.2)
plt.xlabel("alpha")
plt.ylabel("tau_alpha")
plt.show()

In [ ]:
az.plot_parallel(trace, var_names=["alpha_nbd"])
plt.show()